# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Generate-pre-processed-FASTA-files" data-toc-modified-id="Generate-pre-processed-FASTA-files-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Generate pre-processed FASTA files</a></div><div class="lev2 toc-item"><a href="#Setup" data-toc-modified-id="Setup-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Setup</a></div><div class="lev2 toc-item"><a href="#Prepare-low-spike" data-toc-modified-id="Prepare-low-spike-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Prepare low-spike</a></div><div class="lev1 toc-item"><a href="#Simulate-MS-data" data-toc-modified-id="Simulate-MS-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Simulate MS-data</a></div><div class="lev2 toc-item"><a href="#Prepare-quantity-matrix" data-toc-modified-id="Prepare-quantity-matrix-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Prepare quantity matrix</a></div>

# Generate pre-processed FASTA files

Using the custom-made `seqtk generate_spikein.py` script.

Dependencies:

* OpenMS binaries in PATH (version 2.2.0)

## Setup

In [1]:
threads=6
run=batch12_4fold
mkdir ${run}

In [29]:
# sample_names_low="l1 l2 l3"
# sample_names_high="h1 h2 h3"
# samples="${sample_names_low} ${sample_names_high}"

In [30]:
names_base="a b c d"
names_numbers="1 2 3"
sample_names=""
file_names=""

echo -e "name\tbiorepgroup\ttechrepgroup\tcondition" > ${run}/design.tsv

for base in ${names_base}; do
    for nbr in ${names_numbers}; do
        sample_name="${base}${nbr}"
        sample_names="${sample_names} ${sample_name}"
        file_names="${file_names},${sample_name}.fa"
        
        echo -e "${sample_name}\t${nbr}\t1\t${base}" >> ${run}/design.tsv
    done
done

sample_names=$(echo ${sample_names} | sed "s/^ //")
file_names=$(echo ${file_names} | sed "s/^,//")

samples=${sample_names}

echo ${sample_names}
echo ${file_names}

a1 a2 a3 b1 b2 b3 c1 c2 c3 d1 d2 d3
a1.fa,a2.fa,a3.fa,b1.fa,b2.fa,b3.fa,c1.fa,c2.fa,c3.fa,d1.fa,d2.fa,d3.fa


In [31]:
# echo -e "l1\t1\t1\tlow" >> ${run}/design.tsv
# echo -e "l2\t2\t1\tlow" >> ${run}/design.tsv
# echo -e "l3\t3\t1\tlow" >> ${run}/design.tsv
# echo -e "h1\t1\t1\thigh" >> ${run}/design.tsv
# echo -e "h2\t2\t1\thigh" >> ${run}/design.tsv
# echo -e "h3\t3\t1\thigh" >> ${run}/design.tsv

cat ${run}/design.tsv

name	biorepgroup	techrepgroup	condition
a1	1	1	a
a2	2	1	a
a3	3	1	a
b1	1	1	b
b2	2	1	b
b3	3	1	b
c1	1	1	c
c2	2	1	c
c3	3	1	c
d1	1	1	d
d2	2	1	d
d3	3	1	d


## Prepare low-spike

a1 a2 a3 b1 b2 b3 c1 c2 c3 d1 d2 d3
a1.fa,a2.fa,a3.fa,b1.fa,b2.fa,b3.fa,c1.fa,c2.fa,c3.fa,d1.fa,d2.fa,d3.fa


In [24]:
lfqtk generate_spikein_set \
    --background_fa data/uniprot_ecoli.pure.fasta \
    --spikein_fa data/uniprot_potato.pure.fasta \
    --offset_mean 0 \
    --offset_std 0 \
    --base_int 1000000 \
    --noise_std 500000 \
    --back_count 20 \
    --spike_count 5 \
    --spike_folds "1,1,1,1.5,1.5,1.5,2,2,2,2.5,2.5,2.5" \
    --out_base ${run} \
    --sample_names ${file_names} \
    --verbose

4436 entries loaded from data/uniprot_ecoli.pure.fasta as background
403 entries loaded from data/uniprot_potato.pure.fasta as spike-in
20 entries picked as background, 5 as spike-in
25 entries written to batch12_4fold/a1.fa
20 entries picked as background, 5 as spike-in
25 entries written to batch12_4fold/a2.fa
20 entries picked as background, 5 as spike-in
25 entries written to batch12_4fold/a3.fa
20 entries picked as background, 5 as spike-in
25 entries written to batch12_4fold/b1.fa
20 entries picked as background, 5 as spike-in
25 entries written to batch12_4fold/b2.fa
20 entries picked as background, 5 as spike-in
25 entries written to batch12_4fold/b3.fa
20 entries picked as background, 5 as spike-in
25 entries written to batch12_4fold/c1.fa
20 entries picked as background, 5 as spike-in
25 entries written to batch12_4fold/c2.fa
20 entries picked as background, 5 as spike-in
25 entries written to batch12_4fold/c3.fa
20 entries picked as background, 5 as spike-in
25 entries writt

# Simulate MS-data

Generate the OpenMS tool `MSSimulator`.

In [25]:
time parallel -j ${threads} \
"echo \"Processing {}\"
MSSimulator \
    -in ${run}/{}.fa \
    -out_fm ${run}/{}.ground.featureXML" \
    ::: ${samples}
#     -out_id ${run}/{}.ground.idXML \
#     -out ${run}/{}.mzML \
#     -out_pm ${run}/{}.centroided.mzML" \


Processing a3
Loading sequence data from batch12_4fold/a3.fa ...
done (25 protein(s) loaded)
Starting simulation
2017/11/06, 15:53:08: Digest Simulation ... started
2017/11/06, 15:53:08: RT Simulation ... started
2017/11/06, 15:53:10: Predicting RT ... done
2017/11/06, 15:53:10: RT prediction gave 'invalid' results for 740 peptide(s), making them unobservable.
2017/11/06, 15:53:10:   (List is too big to show)
2017/11/06, 15:53:10: Creating experiment with #501 scans ... done
2017/11/06, 15:53:10: Detectability Simulation ... started
2017/11/06, 15:53:10: Ionization Simulation ... started
esi_impurity_probabilities_[0]: 1
weights[0]: 10
2017/11/06, 15:53:10: Simulating 836 features
Progress of 'Ionization':
-- done [took 5.52 s (CPU), 5.52 s (Wall)] -- 
2017/11/06, 15:53:15: #Peptides not ionized: 0
2017/11/06, 15:53:15: #Peptides outside mz range: 283
2017/11/06, 15:53:15: Raw MS1 Simulation ... started
2017/11/06, 15:53:15:   Simulating signal for 2322 features ...
Progress of 'RawMSS

2017/11/06, 15:53:21: Contaminants out-of-RT-range: 204 / 486
2017/11/06, 15:53:21: Contaminants out-of-MZ-range: 111 / 486
2017/11/06, 15:53:21: Compressed data to grid ... 4340689 --> 4097883 (94%)
2017/11/06, 15:53:21: Adding white noise to spectra ...
2017/11/06, 15:53:21: Adding detector noise to spectra ...
2017/11/06, 15:53:21: Detector noise was disabled.
2017/11/06, 15:53:21: Tandem MS Simulation ... disabled
2017/11/06, 15:53:21: Final number of simulated features: 2323
2017/11/06, 15:53:21: Simulation took 13.057616 seconds
2017/11/06, 15:53:21: Storing simulated features in: batch12_4fold/a2.ground.featureXML
2017/11/06, 15:53:22: MSSimulator took 13.29 s (wall), 13.15 s (CPU), 0.00 s (system), 13.15 s (user).
Processing b1
Loading sequence data from batch12_4fold/b1.fa ...
done (25 protein(s) loaded)
Starting simulation
2017/11/06, 15:53:08: Digest Simulation ... started
2017/11/06, 15:53:08: RT Simulation ... started
2017/11/06, 15:53:09: Predicting RT ... done
2017/11/06

2017/11/06, 15:53:22:   (List is too big to show)
2017/11/06, 15:53:22: Creating experiment with #501 scans ... done
2017/11/06, 15:53:22: Detectability Simulation ... started
2017/11/06, 15:53:22: Ionization Simulation ... started
esi_impurity_probabilities_[0]: 1
weights[0]: 10
2017/11/06, 15:53:22: Simulating 835 features
Progress of 'Ionization':
-- done [took 6.73 s (CPU), 6.74 s (Wall)] -- 
2017/11/06, 15:53:29: #Peptides not ionized: 0
2017/11/06, 15:53:29: #Peptides outside mz range: 278
2017/11/06, 15:53:29: Raw MS1 Simulation ... started
2017/11/06, 15:53:29:   Simulating signal for 2319 features ...
Progress of 'RawMSSignal':
-- done [took 3.17 s (CPU), 3.18 s (Wall)] -- 
2017/11/06, 15:53:33: Contaminants out-of-RT-range: 204 / 486
2017/11/06, 15:53:33: Contaminants out-of-MZ-range: 111 / 486
2017/11/06, 15:53:33: Compressed data to grid ... 4318704 --> 4072956 (94%)
2017/11/06, 15:53:33: Adding white noise to spectra ...
2017/11/06, 15:53:33: Adding detector noise to spect

In [26]:
for xml in ${run}/*.featureXML; do 
    ProteinQuantifier \
        -in ${xml} \
        -peptide_out ${xml%.*}.csv
done


Processing summary - number of...
...features: 2323 used for quantification, 2323 total (0 no annotation, 0 ambiguous annotation)
...peptides: 833 quantified, 833 identified (considering best hits only)
ProteinQuantifier took 0.54 s (wall), 0.53 s (CPU), 0.00 s (system), 0.53 s (user).

Processing summary - number of...
...features: 2323 used for quantification, 2323 total (0 no annotation, 0 ambiguous annotation)
...peptides: 833 quantified, 833 identified (considering best hits only)
ProteinQuantifier took 0.55 s (wall), 0.52 s (CPU), 0.00 s (system), 0.52 s (user).

Processing summary - number of...
...features: 2322 used for quantification, 2322 total (0 no annotation, 0 ambiguous annotation)
...peptides: 833 quantified, 833 identified (considering best hits only)
ProteinQuantifier took 0.55 s (wall), 0.52 s (CPU), 0.00 s (system), 0.52 s (user).

Processing summary - number of...
...features: 2323 used for quantification, 2323 total (0 no annotation, 0 ambiguous annotation)
...pe

We receive identical peptide setup due to same protein being picked and cleaved (even if intensities varies).

## Prepare quantity matrix

In [32]:
python3 util_scripts/combine_quant_pd.py \
    --dfs ${run}/*.csv \
    --out_fp ${run}/full_quant.tsv

In [27]:
# header="peptide\tprotein"
# cut_string="1,2"

# file_nbr=0
# for sample in ${samples}; do
#     ((file_nbr=file_nbr+1))
#     header="${header}\t${sample}"    
#     ((abund_col=file_nbr*5))
#     cut_string="${cut_string},${abund_col}"    
# done

# echo ${header}
# echo ${cut_string}

# echo -e ${header} > ${run}/full_quant.tsv
# paste ${run}/*.ground.csv | grep -v "^#" | tail -n +2 \
#     | tr -d "\"" | cut -f ${cut_string} \
#     >> ${run}/full_quant.tsv

peptide\tprotein\ta1\ta2\ta3\tb1\tb2\tb3\tc1\tc2\tc3\td1\td2\td3
1,2,5,10,15,20,25,30,35,40,45,50,55,60


In [33]:
wc -l ${run}/full_quant.tsv
head ${run}/full_quant.tsv

825 batch12_4fold/full_quant.tsv
peptide	protein	a1	a2	a3	b1	b2	b3	c1	c2	c3	d1	d2	d3	h1	h2	h3	l1	l2	l3
FCR	sp|P0A7T7|RS18_ECOLI	216508408.0	105302604.0	188390000.0	124128396.0	61146200.0	26236550.0	62920800.0	152758200.0	37377100.0	102853200.0	209932992.0	132222500.0	136022196.0	133833704.0	133498496.0	136167600.0	131757400.0	138470296.0
FCQR	sp|P39357|YJHF_ECOLI	197866800.0	206806500.0	91024800.0	188364200.0	169931000.0	11359220.0	100704500.0	16145970.0	170408292.0	171960200.0	272123292.0	207982900.0	130084900.0	140282600.0	137579700.0	137107700.0	134154800.0	132605700.0
KFCR	sp|P0A7T7|RS18_ECOLI	80551000.0	38286910.0	69663070.0	44561130.0	24265950.0	9986070.0	35448080.0	74736150.0	18970640.0	50874540.0	65054100.0	40677270.0	63451290.0	64289240.0	62992030.0	65077360.0	61875430.0	66261870.0
AFVLR	sp|P0ADY1|PPID_ECOLI	19200980.0	90075804.0	165011000.0	173858500.0	134742100.0	115509804.0	93534300.0	207598108.0	164900600.0	115195596.0	90998196.0	39363300.0	111659996.0	111465996.0	12703940